<font size="5">Power Measurement of the Akida Model</font>

This notebook was run on the development kit from BrainChip, in order to evaluate the inference an power consumption of the abnormal EEG-signal detection. 

In [1]:
import os
from os import listdir
from os.path import isdir, join
import pathlib

import akida
from akida import FullyConnected
from akida import evaluate_sparsity
import cnn2snn
from cnn2snn import check_model_compatibility
from cnn2snn import quantize
from cnn2snn import quantize_layer
from cnn2snn import convert

from math import ceil

from timeit import default_timer as timer
import numpy as np

In [2]:
# Load quantized model

quantized_model = cnn2snn.load_quantized_model('quantized4_model_raw_eeg.h5')

In [3]:
#  Convert to an Akida model

akida_model = convert(quantized_model)
akida_model.summary()

                  Model Summary                  
_________________________________________________
Input shape     Output shape  Sequences  Layers
[22, 15000, 1]  [1, 1, 2]     1          3     
_________________________________________________

              SW/conv2d_4-dense_2 (Software)               
___________________________________________________________
Layer (type)           Output shape     Kernel shape     
conv2d_4 (InputConv.)  [14976, 22, 40]  (25, 1, 1, 40)   
___________________________________________________________
conv2d_5 (Conv.)       [994, 12, 40]    (1, 11, 40, 40)  
___________________________________________________________
dense_2 (Fully.)       [1, 1, 2]        (1, 1, 477120, 2)
___________________________________________________________



In [4]:
# Define directory

feature_sets_filename = 'final1_eeg_data.npz'

feature_sets = np.load(feature_sets_filename)

print('Feature Sets: ', feature_sets.files)

Feature Sets:  ['x', 'y']


In [5]:
# Assign feature sets

x_test = feature_sets['x']
y_test = feature_sets['y']
x_test = x_test[:10]
y_test = y_test[:10]

In [6]:
# Add dimension

x_test = x_test.reshape(x_test.shape[0], 
                        x_test.shape[1], 
                        x_test.shape[2], 
                        1)

print(x_test.shape)

(10, 22, 15000, 1)


In [7]:
# Show devices

devices = akida.devices()
print('Akida NP available :', devices[0])

Akida NP available : <akida.core.HardwareDevice object at 0xffff2850b3b0>


In [8]:
print(devices[0].version)

BC.00.000.002


In [9]:
# Map Akida model to hardware

device = devices[0]

akida_model.map(device)

akida_model.summary()

                  Model Summary                  
_________________________________________________
Input shape     Output shape  Sequences  Layers
[22, 15000, 1]  [1, 1, 2]     1          3     
_________________________________________________

              SW/conv2d_4-dense_2 (Software)               
___________________________________________________________
Layer (type)           Output shape     Kernel shape     
conv2d_4 (InputConv.)  [14976, 22, 40]  (25, 1, 1, 40)   
___________________________________________________________
conv2d_5 (Conv.)       [994, 12, 40]    (1, 11, 40, 40)  
___________________________________________________________
dense_2 (Fully.)       [1, 1, 2]        (1, 1, 477120, 2)
___________________________________________________________



<font size="5">Default</font>

In [10]:
device.soc.power_measurement_enabled = True

In [11]:
start = timer()
akida_model.forward(x_test)
end = timer()
# Display floor current
floor_power = device.soc.power_meter.floor
print(f'Floor power: {floor_power:.2f} mW')
print(akida_model.statistics)
print(f'Akida inference {len(x_test)} MFCC took {end-start:.2f} s.\n')

Floor power: 908.94 mW

Sequence SW/conv2d_4-dense_2
Average framerate = 0.71 fps
Last inference power range (mW):  Avg 908.46 / Min 908.00 / Max 910.00 / Std 0.85 
Last inference energy consumed (mJ/frame): 1281.66
Akida inference 10 MFCC took 14.11 s.



<font size="5">Clock Mode: Performance</font>

In [12]:
device.soc.clock_mode = akida.soc.ClockMode.Performance
print(device.soc.clock_mode)

ClockMode.Performance


In [13]:
start = timer()
akida_model.forward(x_test)
end = timer()
# Display floor current
floor_power = device.soc.power_meter.floor
print(f'Floor power: {floor_power:.2f} mW')
print(akida_model.statistics)
print(f'Akida inference {len(x_test)} MFCC took {end-start:.2f} s.\n')

Floor power: 908.94 mW

Sequence SW/conv2d_4-dense_2
Average framerate = 0.73 fps
Last inference power range (mW):  Avg 908.73 / Min 908.00 / Max 911.00 / Std 1.14 
Last inference energy consumed (mJ/frame): 1244.14
Akida inference 10 MFCC took 13.69 s.



<font size="5">Clock Mode: Economy</font>

In [14]:
device.soc.clock_mode = akida.soc.ClockMode.Economy
print(device.soc.clock_mode)

ClockMode.Economy


In [15]:
start = timer()
akida_model.forward(x_test)
end = timer()
# Display floor current
floor_power = device.soc.power_meter.floor
print(f'Floor power: {floor_power:.2f} mW')
print(akida_model.statistics)
print(f'Akida inference {len(x_test)} MFCC took {end-start:.2f} s.\n')

Floor power: 742.85 mW

Sequence SW/conv2d_4-dense_2
Average framerate = 0.72 fps
Last inference power range (mW):  Avg 743.45 / Min 742.00 / Max 744.00 / Std 0.90 
Last inference energy consumed (mJ/frame): 1035.63
Akida inference 10 MFCC took 13.93 s.



<font size="5">Clock Mode: LowPower</font>

In [16]:
device.soc.clock_mode = akida.soc.ClockMode.LowPower
print(device.soc.clock_mode)

ClockMode.LowPower


In [17]:
start = timer()
akida_model.forward(x_test)
end = timer()
# Display floor current
floor_power = device.soc.power_meter.floor
print(f'Floor power: {floor_power:.2f} mW')
print(akida_model.statistics)
print(f'Akida inference {len(x_test)} MFCC took {end-start:.2f} s.\n')

Floor power: 662.30 mW

Sequence SW/conv2d_4-dense_2
Average framerate = 0.67 fps
Last inference power range (mW):  Avg 662.24 / Min 662.00 / Max 664.00 / Std 0.66 
Last inference energy consumed (mJ/frame): 991.58
Akida inference 10 MFCC took 14.98 s.

